In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import tensorflow as tf
import malaya_speech
import malaya_speech.train
from malaya_speech.train.model import tacotron2_nvidia as tacotron2
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
input_ids = tf.placeholder(tf.int32, [None, None])
input_lengths = tf.placeholder(tf.int32, [None])
mel_outputs = tf.placeholder(tf.float32, [None, None, 80])
mel_lengths = tf.placeholder(tf.int32, [None])

In [5]:
model = tacotron2.Model([input_ids, input_lengths], [mel_outputs, mel_lengths], 20, prenet_dropout = 0.2)



Instructions for updating:
Use `tf.keras.layers.Conv1D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.batch_normalization` documentation).
Instructions for updating:
Use keras.layers.dropout instead.
Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Please use `layer.add_weight` method instead.
Instructions for u

In [6]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [7]:
model.encoder_logits

{'outputs': <tf.Tensor 'tacotron2_encoder/concat:0' shape=(?, ?, 512) dtype=float32>,
 'src_length': <tf.Tensor 'tacotron2_encoder/floordiv_2:0' shape=(?,) dtype=int32>}

In [8]:
model.decoder_logits

{'outputs': [<tf.Tensor 'tacotron_2_decoder/decoder_1/output_proj/BiasAdd:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/add:0' shape=(?, ?, 80) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/transpose_1:0' shape=(?, ?, ?) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/Sigmoid:0' shape=(?, ?, 1) dtype=float32>,
  <tf.Tensor 'tacotron_2_decoder/decoder/while/Exit_15:0' shape=(?,) dtype=int32>,
  <tf.Tensor 'tacotron_2_decoder/zeros:0' shape=(?, ?, ?) dtype=float32>],
 'stop_token_prediction': <tf.Tensor 'tacotron_2_decoder/decoder_1/stop_token_proj/BiasAdd:0' shape=(?, ?, 1) dtype=float32>}

In [9]:
guided = tacotron2.generate_guided_attention(mel_lengths[0], input_lengths[0])
guided = tf.expand_dims(guided, 0)
guided

<tf.Tensor 'ExpandDims:0' shape=(1, ?, ?) dtype=float32>

In [10]:
from malaya_speech.train.loss import calculate_2d_loss, calculate_3d_loss

In [11]:
r = model.decoder_logits['outputs']
decoder_output, post_mel_outputs, alignment_histories, stop_token_predictions, _, _ = r
stop_token_predictions = stop_token_predictions[:,:,0]

In [12]:
binary_crossentropy = tf.keras.losses.BinaryCrossentropy(
    from_logits=True
)
mse = tf.keras.losses.MeanSquaredError()
mae = tf.keras.losses.MeanAbsoluteError()
mel_loss_before = calculate_3d_loss(
    mel_outputs, decoder_output, loss_fn=mae
)
mel_loss_after = calculate_3d_loss(
    mel_outputs, post_mel_outputs, loss_fn=mae
)
max_mel_length = tf.reduce_max(mel_lengths)
stop_gts = tf.expand_dims(
    tf.range(tf.reduce_max(max_mel_length), dtype=tf.int32), 0
)
stop_gts = tf.tile(
    stop_gts, [tf.shape(mel_lengths)[0], 1]
)
stop_gts = tf.cast(
    tf.math.greater_equal(stop_gts, tf.expand_dims(mel_lengths, 1)),
    tf.float32,
)
stop_token_loss = calculate_2d_loss(
    stop_gts, stop_token_predictions, loss_fn=binary_crossentropy
)
attention_masks = tf.cast(
    tf.math.not_equal(guided, -1.0), tf.float32
)
loss_att = tf.reduce_sum(
    tf.abs(alignment_histories * guided) * attention_masks,
    axis=[1, 2],
)
loss_att /= tf.reduce_sum(attention_masks, axis=[1, 2])

In [13]:
loss_att

<tf.Tensor 'truediv_3:0' shape=(?,) dtype=float32>

In [14]:
loss = stop_token_loss + mel_loss_before + mel_loss_after + loss_att

In [15]:
stop_token_loss, mel_loss_before, mel_loss_after, loss_att

(<tf.Tensor 'binary_crossentropy/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'mean_absolute_error/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'mean_absolute_error_1/weighted_loss/value:0' shape=() dtype=float32>,
 <tf.Tensor 'truediv_3:0' shape=(?,) dtype=float32>)

In [16]:
# r = model.inference(input_ids, input_lengths, speaker_ids)
# r

In [17]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [18]:
sess.run(decoder_output, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]}).shape

(1, 50, 80)

In [19]:
sess.run(alignment_histories, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]}).shape

(1, 9, 50)

In [20]:
sess.run(r, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

[array([[[-0.01520267,  0.00942976,  0.11127456, ..., -0.10087481,
           0.14783642, -0.32963744],
         [-0.01953189,  0.01830612,  0.11961078, ..., -0.09985455,
           0.14657295, -0.33232576],
         [-0.0194759 ,  0.02812577,  0.13153401, ..., -0.09322856,
           0.13459894, -0.34036326],
         ...,
         [-0.04815116,  0.12092838,  0.1606623 , ..., -0.01437848,
           0.02917907, -0.46644217],
         [ 0.00597877,  0.15749355,  0.16767943, ..., -0.08582533,
          -0.02309576, -0.44179893],
         [ 0.00370612,  0.07563721,  0.16179103, ..., -0.04420746,
           0.00747654, -0.4664501 ]]], dtype=float32),
 array([[[-1.6325318 , -0.83928454, -0.70805544, ...,  0.25633737,
           0.97012985, -0.32963744],
         [-0.01953189,  0.01830612,  0.44861582, ..., -0.09985455,
           1.8160677 ,  0.99625176],
         [-0.0194759 ,  0.02812577,  1.1039581 , ...,  0.736087  ,
          -0.91249514, -4.0964627 ],
         ...,
         [-0.04815

In [21]:
sess.run(loss, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]})

array([3.0220482], dtype=float32)

In [22]:
sess.run(loss, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9],
                                       [1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9, 9],
                        mel_outputs: np.random.normal(size=(2, 50, 80)).astype(np.float32),
                        mel_lengths: [50, 50]})

array([3.0477755, 3.0472908], dtype=float32)

In [23]:
sess.run(guided, feed_dict = {input_ids: [[1, 2, 3, 4, 5, 6, 7, 8, 9]],
                        input_lengths: [9],
                        mel_outputs: np.random.normal(size=(1, 50, 80)).astype(np.float32),
                        mel_lengths: [50]}).shape

(1, 9, 50)

In [28]:
model.decoder.params

{'zoneout_prob': 0.0,
 'dropout_prob': 0.1,
 'attention_type': 'location',
 'attention_layer_size': 128,
 'attention_bias': True,
 'decoder_cell_units': 1024,
 'decoder_cell_type': tensorflow.python.ops.rnn_cell_impl.LSTMCell,
 'decoder_layers': 2,
 'enable_prenet': True,
 'prenet_layers': 2,
 'prenet_units': 256,
 'enable_postnet': True,
 'postnet_keep_dropout_prob': 0.7,
 'postnet_data_format': 'channels_last',
 'postnet_conv_layers': [{'kernel_size': [5],
   'stride': [1],
   'num_channels': 512,
   'padding': 'SAME',
   'activation_fn': <function tensorflow.python.ops.gen_math_ops.tanh(x, name=None)>},
  {'kernel_size': [5],
   'stride': [1],
   'num_channels': 512,
   'padding': 'SAME',
   'activation_fn': <function tensorflow.python.ops.gen_math_ops.tanh(x, name=None)>},
  {'kernel_size': [5],
   'stride': [1],
   'num_channels': 512,
   'padding': 'SAME',
   'activation_fn': <function tensorflow.python.ops.gen_math_ops.tanh(x, name=None)>},
  {'kernel_size': [5],
   'stride': [1